Welcome!
--------

Open a terminal and telnet to the ip address on the board:

`telnet ip_address_from_the_board 1234`

Let a helper know if you'd like help.

Once you've connected please put a green sticky note on your laptop so I can see it.

If you have questions at any point (including while I'm talking) feel free to raise your hand or ask a question in this terminal you just opened.

If you're using Windows and telnet isn't installed, ask for help on how to install it, see http://windows.microsoft.com/en-us/windows/telnet-faq#1TC=windows-7

Sticky Notes
===

Red means *"I want help"* or *"Slow down"*
---

Green means *"I'm finished"* or *"I understand"*
---



* Put up a red sticky note if you've got a question and want to talk to a helper about it.
* Put up a green sticky note when we're doing an exercise and you've finished.
* Put up a red or green sticky note when I ask how everyone feels about something to help me know whether we should keep working on something or move on.

Helpers
-------


Notes
---

ballingt.com/

Activity: Open an interactive Python interpreter and square 1492
===

Put a **green sticky** up when you know what 1492 squared is using an interactive Python interpreter.

I reccommend using IPython, ptpython, or bpython instead of the default interpreter, but if you don't already have one of those installed save that for another time.

* [IPython](http://ipython.org/install.html) - I wrote my slides with this
* [ptpython](https://github.com/jonathanslenders/ptpython)
* [bpython](http://bpython-interpreter.org/) - what I'm using when I live-code

Part 1: The Internet
===

What was the internet like in the eighties? Email! BBS

* 1973 - users of mainframes can send each other messages
* 1974 - TCP
* 1984 - bbs
* 1988 - irc
* 1993 - static html pages
* 1995 - server-side scripting
* 1997 - cgi
* 2000 - Ajax
* 2011 - websockets

How do computers talk to each other?
===

It's really interesting but I'm not going to bring it up
---

If you ask a question about it I'll answer it, and I hope you do.

We will talk about this later, but it will make more sense once we're doing it.

But more relevant to us as programmers is...

How do we use Python to tell our computers to talk to each other?
===

Exercise - connect to the chat server with a socket in Python.
---

~~~
import socket
s = socket.socket()
s.connect(('ipaddress', 1234))
s.send(b'hello')
~~~

Note on bytestrings vs strings:

Python 3
--------
* `b"asdf"` is a bytestring
* `"asdf"` is a string (a "unicode" string)

Python 2
--------
`"asdf"` is a bytestring already

By "bytestring" I mean a series of numbers between 0 and 255:
    
    #/usr/bin/python2
    >>> ord('a')
    97
    >> 

TCP Sockets
---
* bidirectional
* send bytestrings
* single use (one connection per socket)

Questions?

**What's going on here?**
Your computer makes radio waves that are interpreter by a router, which reads the first few bytes of them and decides where to send them based on rules to do with the number

**What is a socket object?**
It's similar to a file object in that it's a thing with methods that lets us make requests of the operating system

**What's an ip address?**
IP addresses.

**What is a port?**
An apartment number - the mail service gets this to the right computer via the ip address, but once it gets there there still might be multiple people 

**Internal/external IP addresses?**
There aren't enough IP addresses (256^4) for everyone to have one, so the ip address you have right now isn't actually unique. We'll talk about this later.

Exercise: Write a chat client
--

Reach goal for more experienced folks: use select on stdin and the socket or threads to block on both, printing incoming messages in real time.

Useful functions:
* socket.socket()
* socket.recv()
* input() (raw_input() in Python 2)
* socket.send()
* while loop
* Python 3: .encode() method on strings

In [15]:
#The minimum implementation on the projector will be a while loop,
#alternately sending a line of text if line isn’t blank
#and printing bytes received from a socket.

import socket

def main():
    s = socket.socket()
    s.connect(('123.456.789.012', 1234))
    while True:
        message_to_send = raw_input('msg: ')
        if message_to_send:
            s.send(message_to_send)
        print(s.recv(1000))
    

Ideas for more advanced versions:

* two programs: one that listens, one that sends
* use threads for concurrency
* use select for concurrency
* nicer prompting
* format user input


How could we make our chat server better?



TODO add screencasted demo of sending email over a plaintext (too error-prone to use as an exercise, but very cool) and link to a resource on how to do this)

Protocols
=========

rules about what bytes to send



What's a protcol? A set of rules. We dont' have usernames on the chat. What if we all agreed to do this? Please change your client to write your name, then a colon, then your message.

What's really going on?
--

(Wireshark interlude)
https://www.wireshark.org/

Resources:

* Implementing simple network protocols: http://blog.annharter.com/2015/07/15/three-dead-protocols.html
* More about HTTP: http://www.harihareswara.net/hidden-features-in-http-gwo-2016.pdf


The Web
=======

* HTTP
* HTML

Exercise: Manual http requests/responses with Python sockets.
--------

~~~
>>> import socket
>>> s = socket.socket()
>>> s.connect((‘www.google.com’, 80))
>>> s.send(b‘GET /\n\n’)
>>> s.recv(10000)
~~~

We just tried a new protocol: HTTP

#from https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Example_session
~~~
    GET /index.html HTTP/1.1
    Host: www.example.com
    
(with an empty line at the end)
~~~


#from https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Example_session
~~~
HTTP/1.1 200 OK
Date: Mon, 23 May 2005 22:38:34 GMT
Content-Type: text/html; charset=UTF-8
Content-Encoding: UTF-8
Content-Length: 138
Last-Modified: Wed, 08 Jan 2003 23:11:55 GMT
Server: Apache/1.3.3.7 (Unix) (Red-Hat/Linux)
ETag: "3f80f-1b6-3e1cb03b"
Accept-Ranges: bytes
Connection: close

<html>
<head>
  <title>An Example Page</title>
</head>
<body>
  Hello World, this is a very simple HTML document.
</body>
</html>
~~~

Project: write an http client
------

* read the site from command line arguments or raw_input
* download it and put it in a file
* just put the content in a file, not the whole response

The server side of sockets
---

So sockets are a conversation with the operating system? We can ask it some other things.

server code
~~~
import select
import socket

server = socket.socket()
#server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind(('', 1234))
server.listen(5)

clients = []

while True:
    ready_to_read, _, _ = select.select(clients + [server], [], [])
    for r in ready_to_read:
        if r is server:
            s, addr = server.accept()
            clients.append(s)
        else:
            msg = r.recv(1000)
            for client in clients:
                if client is not r:
                    client.send(msg)
~~~


Project: Write a server, have your neighbor connect to it
---



~~~
import socket

server = socket.socket()
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind(('', 1234))
server.listen(1)

while True:
    s, addr = server.accept()
    msg = s.recv(1000)
    s.send(msg.upper())
~~~

Project - make file-serving web server implement http 0.9
---

https://www.w3.org/Protocols/HTTP/AsImplemented.html

we’ll step though it’s shortcomings: content type, parsing the rest of the request, caching, etc.

Reach goals for more experienced folks will be to compress the resource. We’ll allow 15 minutes.

~~~
import socket

listener = socket.socket()
listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
listener.bind(('', 8000))
listener.listen(5)

while True:
    s, addr = listener.accept()
    print 'server received connection from', addr
    request = s.recv(10000)
    print 'request we received:', request
    method, rest = request.split(' ', 1)
    path, rest = rest.split(None, 1)
    s.send('HTTP/0.9 200 OK\n\n')
    with open('.'+path) as f:
        s.sendall(f.read())
    s.close()
~~~

This is a tough one, there's a lot of ways to mess up. We should go over the HTTP protocol.

Go over tools used to test this: show browser request in terminal, and show Google response in terminal.

15 minute break
==


Learning goal: “OMG servers aren’t magic, also practice with Python filesystem APIs”

Part 3: CGI & WSGI
Giving people information you don't have

Exercise - modify file server to return the current time
We’ll encourage people who didn’t get a working version of the previous project to do a git checkout or to copy and paste a gist into a file to start with a working web server. This exercise should take 5 minutes.
Learning goal: “OMG the file path part of a HTTP request is just a suggestion!”

Exercise - simple cgi script (returns a random file or prints some numbers)
We’ll introduce the basics of the cgi protocol, but dealing with query parameters and post data will be reach goals. We’ll allow 10 minutes so people can be creative - this is an exercise everyone should be able to do since it only requires printing text and optiuonally some HTML.

Demo - modifying the script (teaching subprocess management is too much of a distraction)

Lecture - commercial use of the internet backbone permitted in 1995(!), need for more dynamic pages in the late 90s.  cgi scripts, php.  Dot.com boom.  Overview of cgi.

Lecture - wsgi, fast_cgi, speed, organization, web frameworks

Exercise - write a web application WSGI function
After being presented in lecture with something like https://gist.github.com/thomasballinger/5807241 writing a function that calls a callback in it’s body should be the main tricky bit. Should take 5 minutes.

Project - writing a framework: a function that takes a dictionary of paths and responses and returns the appropriate wsgi application
This could go on for a while. The minimum project on the as described will be a function that takes a dictionary and returns a function, but all the piece will be in place to write a flask-like api for people comfortable with decorators - and helpers will all be prepared for people stuck on them.



Interlude: Don’t use sockets! <blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Sockets are the assembly language of network programming. There’s usually a much easier library to use.</p>&mdash; Ned Batchelder (@nedbat) <a href="https://twitter.com/nedbat/status/710144098028666880">March 16, 2016</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>


Part 4: JavaScript, the DOM
The browser is a VM, the HTML is just initialization

Lecture - things added to HTML HTML page can spark off a bunch of requests.
Js, applets, flash,

Exercise - write a chat history backend endpoint used by a JavaScript snippet that repeatedly makes a request and puts the text in a div
This will take at least 10 minutes but could stretch well beyond this as beginners people likely back in their comfort zones writing HTML and CSS.

In [1]:
!pwd

/Users/tomb/Dropbox/code/webbeginning
